<a href="https://colab.research.google.com/github/miramira227/PointNet_pytorch/blob/master/PointNet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Upload**

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! mkdir mtrain
! mkdir mtest
! mkdir shape

mkdir: cannot create directory ‘shape’: File exists


In [ ]:
cd drive/My\ Drive/pointnet_data

/content/drive/My Drive/pointnet_data


In [ ]:
! cp model40jtrain.zip '/content/mtrain'
! cp model40jtest.zip '/content/mtest'
! cp shapej.zip '/content/shape'

In [ ]:
cd /content/mtrain/

/content/mtrain


In [ ]:
! unzip model40jtrain.zip

In [ ]:
cd /content/mtest

/content/mtest


In [ ]:
! unzip model40jtest.zip

In [ ]:
! rm model40jtest.zip 

In [ ]:
cd /content/shape

/content/shape


In [ ]:
! unzip shapej.zip
! rm shapej.zip

#**Check Device Type**##

In [ ]:
import torch 

if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

# check tensor device 
a = torch.randn(1, 2, 3).to(device)
print(f'current device is {a.device}')

current device is cuda:0


#**1. Data Augmentation**#

In [ ]:
import torch 
import numpy as np 

input = torch.randn(2, 5, 3).to(device)

def data_aug(input):
  angle = np.random.uniform(0.0, np.pi * 2)
  rotate_matrix = torch.FloatTensor([[np.cos(angle), -1 * np.sin(angle)], 
                          [np.sin(angle), np.cos(angle)]]).repeat(input.size(0), 1, 1)    # (2, 2) --> (7, 2, 2)
  rotate_matrix = rotate_matrix.to(device)
  # data augmentation along up-axis(suppose z-axis).
  input[:, :, 0:2] = torch.bmm(input[:, :, 0:2], rotate_matrix)
  input = torch.normal(0.0, 0.2, size = input.size()).to(device)
  return input

print(f'original input was \n {input}')
out = data_aug(input)
print(f'output is \n {out}')

original input was 
 tensor([[[-0.1801,  0.0308, -0.3258],
         [ 0.0419,  0.5573, -0.7421],
         [-1.4897, -1.3174, -0.1955],
         [-0.4736,  0.2591, -2.5837],
         [-0.5975, -0.6196,  1.1691]],

        [[-0.2908, -1.0805, -2.0121],
         [ 0.4024,  1.7966,  0.8716],
         [-0.1868,  1.3511, -1.2778],
         [ 0.5074,  0.7624, -2.7151],
         [-0.7577, -0.2963,  1.2320]]], device='cuda:0')
output is 
 tensor([[[-0.0577, -0.0540,  0.1236],
         [-0.0824, -0.0208, -0.1363],
         [ 0.1318, -0.1262,  0.0690],
         [ 0.0821, -0.3246,  0.5202],
         [ 0.1820, -0.1063,  0.0706]],

        [[-0.5081, -0.1779, -0.1136],
         [ 0.1404, -0.3579,  0.2005],
         [ 0.1134,  0.2821,  0.2932],
         [-0.0901, -0.1031, -0.3225],
         [ 0.2648,  0.2162,  0.2515]]], device='cuda:0')


#**2. Transform Netw**#

**(1) input transformer matrix**

* make transformer matrix of size(batch, 3, 3)

In [ ]:
# 1. input transform net 
import torch.nn as nn 
import torch 
import numpy as np 

# suppose input point cloud size is (2, 5, 3) = (batch size, length, dim)
input_pointset = torch.randn((2, 5, 3)).transpose(1, 2).to(device)  

class MiniPNet(nn.Module):
  def __init__(self):
    super(MiniPNet, self).__init__()
    self.conv1 = nn.Conv1d(3, 64, 1)
    self.bn1 = nn.BatchNorm1d(64)
    self.relu = nn.ReLU()

    self.conv2 =nn.Conv1d(64, 128, 1)
    self.bn2 = nn.BatchNorm1d(128)

    self.conv3 = nn.Conv1d(128, 1024, 1)
    self.bn3 = nn.BatchNorm1d(1024)

    self.fc1 = nn.Linear(1024, 512)
    self.bn4 = nn.BatchNorm1d(512)

    self.fc2 = nn.Linear(512, 256)
    self.bn5 = nn.BatchNorm1d(256)

    self.fc3 = nn.Linear(256, 9)    

    with torch.no_grad():
      self.fc3.weight.data.fill_(0)
      self.fc3.bias.data.fill_(0)
      iden = torch.eye(3).view(-1)   # double type is required
      self.fc3.bias.data = self.fc3.bias.data + iden    # += invokes 'requires grad' error 

  def forward(self, x):
    x = self.relu(self.bn1(self.conv1(x)))
    x = self.relu(self.bn2(self.conv2(x)))
    x = self.relu(self.bn3(self.conv3(x)))

    x = torch.max(x, dim=2)[0]    

    x = self.relu(self.bn4(self.fc1(x)))
    x = self.relu(self.bn5(self.fc2(x)))

    x = self.fc3(x)

    x = torch.reshape(x, (-1, 3, 3))
    # print(f'input transformer matrix is {x[0]}')
    return x


net = MiniPNet().to(device)
out = net(input_pointset)
print(f"out size is {out.size()}")

for a, b in zip(net.named_modules(), net.parameters()):
  print(a, b)

print(net.named_modules)

out size is torch.Size([2, 3, 3])
('', MiniPNet(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=9, bias=True)
)) Parameter containing:
tensor([[[ 0.4101],
         [-0.1927],
         [-0.0100]],

        [[ 0.0440],
         [-0.0199]

In [ ]:
# 2. feature transform net
import torch.nn as nn 
import torch 
import numpy as np 


input_pointset = torch.randn((2, 5, 128)).transpose(1, 2).to(device)   

# k = integer related to the shape of input tensor, 
# and also a key factor for the shape of output transformer matrix (batch, k, k)
# which will transform the input tensor

class FeatPNet(nn.Module):
  def __init__(self, k):
    super(FeatPNet, self).__init__()
    self.k = k

    self.conv1 = nn.Conv1d(self.k, 64, 1)
    self.bn1 = nn.BatchNorm1d(64)
    self.relu = nn.ReLU()

    self.conv2 =nn.Conv1d(64, 128, 1)
    self.bn2 = nn.BatchNorm1d(128)

    self.conv3 = nn.Conv1d(128, 1024, 1)
    self.bn3 = nn.BatchNorm1d(1024)

    self.fc1 = nn.Linear(1024, 512)
    self.bn4 = nn.BatchNorm1d(512)

    self.fc2 = nn.Linear(512, 256)
    self.bn5 = nn.BatchNorm1d(256)

    self.fc3 = nn.Linear(256, self.k * self.k)    
    self.fc3.weight.data.fill_(0)
    self.fc3.bias.data.fill_(0)

    iden = torch.eye(self.k).view(-1)
    
    self.fc3.bias.data = self.fc3.bias.data + iden


  def forward(self, x):
    x = self.relu(self.bn1(self.conv1(x)))
    x = self.relu(self.bn2(self.conv2(x)))
    x = self.relu(self.bn3(self.conv3(x)))

    x = torch.max(x, dim=2)[0]    # (batch, 1024)

    x = self.relu(self.bn4(self.fc1(x)))
    x = self.relu(self.bn5(self.fc2(x)))

    x = self.fc3(x)
    x = torch.reshape(x, (-1, self.k, self.k))
    # print(f'transformer matrix is {x[0]}')
    return x

net = FeatPNet(input_pointset.size(1)).to(device) #dimension of input tensor
print(f"input size is {input_pointset.size()}")
out = net(input_pointset)
print(f"output size is {out.size()}")

input size is torch.Size([2, 128, 5])
output size is torch.Size([2, 128, 128])


#**Feature Transformer Regularizer**#

In [ ]:
import torch 

feat_stn = torch.randn((32, 64, 64)).to(device)

def regularizer(feat_stn):
  size = feat_stn.size(1)
  # iden = torch.eye(size).view(-1).repeat(feat_stn.size(0)).view(-1, size, size).to(device)
  iden = torch.eye(size).repeat(feat_stn.size(0), 1, 1).to(device)
  AAT = torch.bmm(feat_stn, feat_stn.transpose(1, 2)).to(device)
    
  norm = 0
  for i in range(iden.size(0)):
    # torch.norm is used to calculate frobius norm of matrix, 
    # which is similar to L2 norm in vector
    fro = torch.norm(iden[i] - AAT[i])    
    norm += fro
  return norm/iden.size(0)

out = regularizer(feat_stn)
print(f"regularizer out value is {out}")


regularizer out value is 717.2933349609375


#**Baseline for Global Feature**#

In [ ]:
import torch.nn as nn 
import torch 
import numpy as np 

input = torch.randn((2, 5, 3)).transpose(1, 2).to(device)

class Baseline(nn.Module):
  def __init__(self, k=64):
    super(Baseline, self).__init__()
    self.mini_T = MiniPNet()

    self.conv1 = nn.Conv1d(3, 64, 1)
    self.bn1 = nn.BatchNorm1d(64)
    self.relu = nn.ReLU()

    self.conv2 = nn.Conv1d(64, 64, 1)
    self.bn2 = nn.BatchNorm1d(64)

    self.feat_T = FeatPNet(k)

    self.conv3 = nn.Conv1d(64, 64, 1)
    self.bn3 = nn.BatchNorm1d(64)

    self.conv4 = nn.Conv1d(64, 128, 1)
    self.bn4 = nn.BatchNorm1d(128)

    self.conv5 = nn.Conv1d(128, 1024, 1)
    self.bn5 = nn.BatchNorm1d(1024)

  def forward(self, x):
    mini_stn = self.mini_T(x)    

    x = torch.bmm(mini_stn, x)

    x = self.relu(self.bn1(self.conv1(x)))
    x = self.relu(self.bn2(self.conv2(x)))

    feat_stn = self.feat_T(x)

    local_feat = torch.bmm(feat_stn, x)

    x = self.relu(self.bn3(self.conv3(local_feat)))
    x = self.relu(self.bn4(self.conv4(x)))
    x = self.relu(self.bn5(self.conv5(x)))

    global_feat = torch.max(x, dim=2)[0]

    return global_feat, feat_stn

net = Baseline().to(device)
global_feat, feat_stn = net(input)
print(f"global_feat size: {global_feat.size()}, \nfeat_stn size: {feat_stn.size()}")

global_feat size: torch.Size([2, 1024]), 
feat_stn size: torch.Size([2, 64, 64])


#**Classification Module**#

In [ ]:
import torch.nn.functional as F

class Classification(nn.Module):
  def __init__(self, num_cls):
    super(Classification, self).__init__()
    self.fc1 = nn.Linear(1024, 512)
    self.bn1 = nn.BatchNorm1d(512)
    
    self.fc2 = nn.Linear(512, 256)
    self.bn2 = nn.BatchNorm1d(256)

    self.dropout = nn.Dropout(0.3)    # rate to be zeroed 
    self.fc3 = nn.Linear(256, num_cls)

    self.relu = nn.ReLU()

  def forward(self, x):
    global_feat, feat_stn = x[0], x[1]
    x = self.relu(self.bn1(self.fc1(global_feat)))
    x = self.relu(self.bn2(self.dropout(self.fc2(x))))
    x = self.relu(self.fc3(x))
    
    cls_score = F.log_softmax(x, dim= 1)
    # print(f'cls_score is {cls_score}')
    return cls_score, feat_stn, global_feat 


num_cls = 40

input = torch.rand(2, 5, 3).transpose(1, 2).to(device)
cls_net = nn.Sequential(Baseline(), Classification(num_cls)).to(device).eval()
cls_score, feat_stn, global_feat = cls_net(input)
# print(f"cls_score size: {cls_score.size()}, \nfeat_stn size: {feat_stn.size()},"
# f"\nglobal_feat size: {global_feat.size()}")

#**Train/Val Split**#

val 결과 나온 weight 먼저 확인하고 

# **Classification Training**

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn 
import json
import torch
import math
import time
import os 

model = nn.Sequential(Baseline(), Classification(40)).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

batch = 32

train_path = '/content/mtrain'
train_data = os.listdir(train_path)

test_path = '/content/mtest'
test_data = os.listdir(test_path)

train_loader = torch.utils.data.DataLoader(train_data, batch, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch, shuffle=True)

train_loss, val_loss = [], []
epochs = 100

now = time.time()
print('training start!')
for epoch in range(epochs):

  running_loss = 0.0

  for bidx, trainb32 in enumerate(train_loader):

    bpts, blabel = [], []
    
    for i, data in enumerate(trainb32):
      path = os.path.join(train_path, data)
      
      with open(path, 'r') as f:
        jdata = json.load(f)
      
      label = jdata['label']
      pts = jdata['pts']

      bpts.append(pts) 
      blabel.append(label)

    bpts = torch.tensor(bpts).transpose(1, 2).to(device)
    blabel = torch.tensor(blabel).to(device)
    
    input = data_aug(bpts).to(device)

    optimizer.zero_grad()

    y_pred, feat_stn, glob_feat = model(input)
    # print(f'global_feat is {global_feat}')
    loss = F.nll_loss(y_pred, blabel) + 0.001 * regularizer(feat_stn)

    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    if bidx % 10 == 9:
      vrunning_loss = 0
      vacc = 0
      model.eval()
      with torch.no_grad():
        
        # val batch
        for vbidx, testb32 in enumerate(test_loader):
          bpts, blabel = [], []
          
          for j, data in enumerate(testb32):
            path = os.path.join(test_path, data)
            
            with open(path, 'r') as f:
              jdata = json.load(f)
            
            label = jdata['label']
            pts = jdata['pts']

            bpts.append(pts) 
            blabel.append(label)

          bpts = torch.tensor(bpts).transpose(1, 2).to(device)
          blabel = torch.tensor(blabel).to(device)
          
          input = data_aug(bpts).to(device)
          vy_pred, vfeat_stn, vglob_feat = model(input)
          # print(f'global_feat is {vglob_feat}')
          vloss = F.nll_loss(vy_pred, blabel) + 0.001 * regularizer(vfeat_stn)
          _, vy_max = torch.max(vy_pred, dim=1) 
          vy_acc = torch.sum(vy_max == blabel) / batch
          
          vacc += vy_acc
          vrunning_loss += vloss

      # print every training 10th batch
      train_loss.append(running_loss / len(train_loader))
      val_loss.append(vrunning_loss / len(test_loader))

      print(f"Epoch {epoch+1}/{epochs} {bidx}/{len(train_loader)}.. "
            f"Train loss: {running_loss / 10:.3f}.."
            f"Val loss: {vrunning_loss / len(test_loader):.3f}.."
            f"Val Accuracy: {vacc/len(test_loader):.3f}.."
            f"Time: {time.time() - now}")
      now = time.time()
      running_loss = 0
      model.train()
      
          
print(f'training finish! training time is {time.time() - now}')
print(model.parameters())

savePath = '/content/modelpath.pth'
torch.save(model.state_dict(), '/content/modelpath.pth')

Epoch 88/100 29/238.. Train loss: 3.410..Val loss: 3.605..Val Accuracy: 0.000..Time: 5.515955686569214
Epoch 88/100 39/238.. Train loss: 3.366..Val loss: 3.590..Val Accuracy: 0.000..Time: 5.55347204208374
Epoch 88/100 49/238.. Train loss: 3.412..Val loss: 3.605..Val Accuracy: 0.000..Time: 5.48908805847168
Epoch 88/100 59/238.. Train loss: 3.437..Val loss: 3.596..Val Accuracy: 0.000..Time: 5.599501848220825
Epoch 88/100 69/238.. Train loss: 3.339..Val loss: 3.602..Val Accuracy: 0.000..Time: 5.51908802986145
Epoch 88/100 79/238.. Train loss: 3.386..Val loss: 3.603..Val Accuracy: 0.000..Time: 5.58879828453064
Epoch 88/100 89/238.. Train loss: 3.417..Val loss: 3.601..Val Accuracy: 0.000..Time: 5.548323392868042
Epoch 88/100 99/238.. Train loss: 3.374..Val loss: 3.607..Val Accuracy: 0.000..Time: 5.6686036586761475
Epoch 88/100 109/238.. Train loss: 3.412..Val loss: 3.600..Val Accuracy: 0.000..Time: 5.619024991989136
Epoch 88/100 119/238.. Train loss: 3.448..Val loss: 3.603..Val Accuracy: 0.

In [ ]:
import torch 

a = torch.tensor([1, 2, 3, 1])
b = torch.tensor([2, 2, 1, 1])
print(torch.sum(a == b))

tensor(2)


In [ ]:
model = nn.Sequential(Baseline(), Classification(40)).to(device)
model.load_state_dict(torch.load('/content/modelpath.pth'))
model.eval()

for param in model.parameters():
  print(param.size(), param)

torch.Size([64, 3, 1]) Parameter containing:
tensor([[[-0.0778],
         [-0.5144],
         [-0.4890]],

        [[-0.2123],
         [-0.0484],
         [ 0.1117]],

        [[ 0.3344],
         [-0.3609],
         [ 0.4029]],

        [[-0.2175],
         [ 0.1201],
         [ 0.6253]],

        [[-0.5256],
         [-0.4006],
         [-0.1874]],

        [[-0.4075],
         [-0.5153],
         [-0.3382]],

        [[-0.0083],
         [-0.5175],
         [-0.4295]],

        [[-0.4127],
         [-0.2899],
         [ 0.3528]],

        [[ 0.0796],
         [ 0.5573],
         [-0.1775]],

        [[-0.1548],
         [-0.1058],
         [ 0.5159]],

        [[-0.0103],
         [ 0.2960],
         [-0.3208]],

        [[ 0.1083],
         [-0.2178],
         [-0.2965]],

        [[ 0.2534],
         [ 0.0459],
         [-0.4102]],

        [[-0.2943],
         [-0.2182],
         [ 0.3456]],

        [[-0.0425],
         [-0.5693],
         [-0.4695]],

        [[-0.5755],
     